In [1]:
import sys
import math
import time
import chars2vec
import numpy as np
import pandas as pd
from textdistance import levenshtein
from annoy import AnnoyIndex
from scipy.spatial import distance

In [2]:
# Helper function for timing
def debug_time(msg, init):
    print(f"{msg} [{(time.time()-init)*1000:.3f}ms]", file=sys.stderr, flush=True)

In [3]:
# Some Constants
NAME_MAXLENGTH = 20
VEC_DIMS = 20 # actually |vec| = VEC_DIMS*k (for some variable k, see function <string2vec>)
NUM_SIMILAR_RESULTS = 20 # get 20 most similar results by default

CYCLE_INPUT_STRING = True  # Should we wrap-around strings if they are shorter than NAME_MAXLENGTH?
                           # I think produces better results to 'control' for differences in string length

# Permissible Distance Metrics
ANNOY_DISTANCE_METRICS = ["angular", "euclidean", "manhattan", "hamming", "dot"]

In [4]:
# Load up 100 dimensional pre-trained chars2vec model
c2v_model = chars2vec.load_model('eng_100')

In [5]:
# Extract out 'License Name' column from dataset
dataset = "list-of-nea-licensed-eating-establishments.csv"
df = pd.read_csv(dataset)
df = df['licensee_name']
df = df.drop_duplicates()

In [6]:
print(df)

0         REPUBLIC HOTELS & RESORTS LIMITED
2                         M.K. RAMA PTE LTD
3             GRAND PARK PROPERTY PTE. LTD.
4                  MILLENIA PRIVATE LIMITED
6              BCH HOTEL INVESTMENT PTE LTD
                        ...                
36682     AINON BTE BADRI ( AINON BTE ALI )
36683         SYED IBRAHIM BIN PEER MOHAMED
36684                      SAITON BINTE ALI
36685                     AMINAH BTE K OMAR
36686    AISHA BEGAM BINTE MOHAMED MUSTHAPA
Name: licensee_name, Length: 22878, dtype: object


In [7]:
# Sanitization function
def sanitize_alpha(data, maxlen=NAME_MAXLENGTH):
    orig_string = ''.join([c for c in data.lower() if c.isalpha()])
    if (CYCLE_INPUT_STRING):
        cyc_string = orig_string*(math.ceil(maxlen/len(orig_string)))
        return cyc_string[:maxlen]
    else:
        return orig_string[:maxlen]

In [8]:
# Clean up dataframe contents to only include ALPHABETICAL letters in lowercase (26 a-z)
clean_df = df.map(sanitize_alpha)
print(clean_df)

0        republichotelsresort
2        mkramapteltdmkramapt
3        grandparkpropertypte
4        milleniaprivatelimit
6        bchhotelinvestmentpt
                 ...         
36682    ainonbtebadriainonbt
36683    syedibrahimbinpeermo
36684    saitonbintealisaiton
36685    aminahbtekomaraminah
36686    aishabegambintemoham
Name: licensee_name, Length: 22878, dtype: object


In [9]:
maxlen_func = np.vectorize(len)
maxlen_func(clean_df).max()

20

In [10]:
# Returns alternate characters in given string
def skipGram(string):
    return string[::2]

# encoding function
def enc_sum(string_arr):
    a = ord('a')
    return int(sum([ord(c)-a for c in string_arr]))

def enc_xor(string_arr):
    a = ord('a')
    return int(accumulate(string_arr, lambda accum,ele: accum^(ord(ele)-a)))

# Specify some naive transform into vector representation for our data
# 3 n-gram xor? of neighbors | 2,4 skip-gram xor? of neighbors
# excess truncated | padded with 0-s
# |vec| = dims*4 (for convenience)
# also dims should >= maxlength of our data!
def string2vec(string, enc_func, dims=VEC_DIMS, debug=False):
    a = ord('a')
    # original string
    orig = [ord(c)-a for c in string][:min(len(string), dims)]
    # 3 n-gram
    ngram = [enc_func(string[i:i+3]) for i in range(min(len(string)-3, dims))]
    # 2 skip-gram
    skip2gram = [enc_func(skipGram(string[i:i+5])) for i in range(min(len(string)-5, dims))]
    # 4 skip-gram
    skip4gram = [enc_func(skipGram(string[i:i+9])) for i in range(min(len(string)-9, dims))]
    if (debug):
        print(ngram,skip2gram,skip4gram)
    concat_data = (orig, ngram, skip2gram, skip4gram)
#     concat_data = (ngram, skip2gram, skip4gram)
#     concat_data = (orig, ngram)
    vector = np.zeros(dims*len(concat_data))
    for d, vec in enumerate(concat_data):
        vector[dims*d:dims*d+len(vec)] = vec
    return vector/np.linalg.norm(vector)
#     return vector

In [11]:
print(string2vec("republichotelsresort", enc_sum, dims=VEC_DIMS, debug=True))

[36, 39, 36, 32, 20, 21, 17, 23, 40, 37, 34, 33, 46, 39, 39, 36, 49] [33, 35, 24, 33, 16, 27, 34, 20, 37, 36, 47, 26, 46, 36, 52] [48, 51, 50, 51, 46, 49, 62, 42, 72, 54, 82]
[0.06099972 0.01435288 0.05382328 0.07176438 0.00358822 0.03947041
 0.02870575 0.00717644 0.02511753 0.05023507 0.06817616 0.01435288
 0.03947041 0.06458794 0.06099972 0.01435288 0.06458794 0.05023507
 0.06099972 0.06817616 0.12917588 0.13994054 0.12917588 0.11482301
 0.07176438 0.0753526  0.06099972 0.08252904 0.14352876 0.1327641
 0.12199945 0.11841123 0.16505807 0.13994054 0.13994054 0.12917588
 0.17582273 0.         0.         0.         0.11841123 0.12558766
 0.08611726 0.11841123 0.0574115  0.09688191 0.12199945 0.07176438
 0.1327641  0.12917588 0.16864629 0.09329369 0.16505807 0.12917588
 0.18658739 0.         0.         0.         0.         0.
 0.17223451 0.18299917 0.17941095 0.18299917 0.16505807 0.17582273
 0.22246958 0.1507052  0.25835177 0.19376382 0.29423396 0.
 0.         0.         0.         0.  

In [12]:
def string2vec_encSum(string):
    return string2vec(string, enc_sum, dims=VEC_DIMS)
enc_time = time.time()
vec_df = clean_df.map(string2vec_encSum)
enc_time = time.time() - enc_time
print(vec_df)
print(f"Encoding {len(vec_df)} vectors: {enc_time*1000:.5f}ms | avg: {enc_time*1000/len(vec_df):.5f}ms")

0        [0.06099972265241056, 0.014352875918214249, 0....
2        [0.045912690759057614, 0.03826057563254801, 0....
3        [0.018799819439045722, 0.053266155077296214, 0...
4        [0.04675529712325957, 0.031170198082173046, 0....
6        [0.0034414692207825072, 0.0068829384415650145,...
                               ...                        
36682    [0.0, 0.03999000374843818, 0.06498375609121206...
36683    [0.08407819170887931, 0.1121042556118391, 0.01...
36684    [0.07300204093126623, 0.0, 0.03244535152500721...
36685    [0.0, 0.0544813303849538, 0.036320886923302535...
36686    [0.0, 0.042171903025219545, 0.0948867818067439...
Name: licensee_name, Length: 22878, dtype: object
Encoding 22878 vectors: 1419.08813ms | avg: 0.06203ms


In [13]:
# More sophisticated vector embedding using chars2vec
enc_time = time.time()
vec_df = c2v_model.vectorize_words(list(clean_df))
enc_time = time.time() - enc_time
print(vec_df)
print(f"Encoding {len(vec_df)} vectors: {enc_time*1000:.5f}ms | avg: {enc_time*1000/len(vec_df):.5f}ms")

[[-5.1368988e-01  3.8920587e-01  5.8130384e-03 ...  2.4472086e-01
   4.0191626e-03  6.0844743e-01]
 [-1.0226086e-01  1.8165192e-01 -6.6250682e-01 ... -3.3742964e-01
   5.3404528e-03  9.3769652e-01]
 [-4.6932191e-01  3.5536280e-01 -4.3260887e-01 ...  2.5469857e-01
   1.9182866e-02  8.5552245e-01]
 ...
 [-7.2969013e-01  7.0039731e-01 -5.3937656e-01 ...  5.3412195e-02
   2.1636357e-04  6.3521457e-01]
 [-2.1015666e-01  5.8237005e-02 -6.2032503e-01 ... -4.8170123e-01
   5.5862684e-04  9.0191376e-01]
 [-1.4618242e-01  7.1757622e-02 -5.9894139e-01 ... -4.3950951e-01
   1.2857503e-03  8.2262570e-01]]
Encoding 22878 vectors: 5372.79081ms | avg: 0.23485ms


In [16]:
# Wrapper Class for Annoy Trees
class AnnoyTree(object):
    def __init__(self, dims, wordlist, vectorlist, distance="euclidean"):
        self.model = None
        self.distance_metric = distance
        self.dims = dims
        self.wordlist = wordlist[:]
        self.vectors = vectorlist[:]
        if (self.distance_metric not in ANNOY_DISTANCE_METRICS):
            print(f"ERROR!! Distance metric specified: {self.distance_metric} NOT FOUND")
    
    # Trees control how many index trees are generated
    # larger => more accurate, larger index file size
    def build(self, output_fname, trees=10):
        b_time = time.time()
        self.model = AnnoyIndex(self.dims, self.distance_metric)
        for i, word in enumerate(self.wordlist):
            self.model.add_item(i, self.vectors[i])
        self.model.build(trees)
        debug_time("Model built in time", b_time)
        self.model.save(output_fname)
        
    # Extract model from saved index file
    def load(self, input_fname):
        self.model = AnnoyIndex(self.dims, self.distance_metric)
        self.model.load(input_fname)
        
    # Interact with a built model
    def query(self, q_vec, search_k=None, num_results=NUM_SIMILAR_RESULTS):
        if (self.model is None):
            print("Error: Model is not yet build or loaded!")
            return None
        q_time = time.time()
        if (search_k is not None):
            neighbor_idx, dist = self.model.get_nns_by_vector(q_vec, num_results, search_k=search_k, include_distances=True)
        else:
            neighbor_idx, dist = self.model.get_nns_by_vector(q_vec, num_results, include_distances=True)
        debug_time("Query in time", q_time)
        neighbors = map(lambda i: self.wordlist[i], neighbor_idx)
        return zip(neighbors, dist)
    
    # Brute-force search
    def brute_force(self, q_vec, num_results=NUM_SIMILAR_RESULTS):
        q_time = time.time()
        neighbor_idx, dist = zip(*sorted([(i, distance.cosine(q_vec, v)) for i,v in enumerate(self.vectors)], key=lambda x: x[1])[:num_results])
        debug_time("Brute-Force in time", q_time)
        neighbors = map(lambda i: self.wordlist[i], neighbor_idx)
        return zip(neighbors, dist)
    
    # Control using levenstein distance
    def control_query(self, q, num_results=NUM_SIMILAR_RESULTS):
        q_time = time.time()
        results = sorted([(w, levenshtein(q, sanitize_alpha(w))) for w in self.wordlist], key=lambda x: x[1])[:num_results]
        debug_time("Brute-Force with levenshtein", q_time)
        return results

In [17]:
test_tree = AnnoyTree(len(vec_df[0]), list(df), list(vec_df))

In [18]:
test_tree.build("test_tree.ann", trees=53)

Model built in time [827.553ms]


In [19]:
q_vec = c2v_model.vectorize_words([sanitize_alpha("tan chee hoon")])[0]
print(q_vec)

[-6.4280587e-01  9.3729451e-02 -8.3516635e-02  3.3228386e-03
  9.2821896e-01 -5.1187521e-01  1.6161662e-01  3.7570089e-01
  1.6440129e-01 -2.6656640e-01  1.0424998e-02 -1.6829653e-01
 -6.4724731e-01  1.8096562e-01 -7.7520795e-03  1.0755547e-01
 -2.6615241e-01  4.8587485e-03  6.5829790e-01  1.7199401e-02
 -3.8973264e-02  1.1518379e-04 -3.5060290e-02 -2.5793332e-01
 -8.0264896e-01  7.4051213e-01  5.5432284e-01  6.9274469e-03
  9.9025995e-02  2.7460076e-02  1.6123313e-03  2.1976231e-01
  4.5882529e-01 -2.0103514e-01  6.2030144e-02 -1.8514501e-01
  7.2602725e-01 -3.5894036e-01  1.4303601e-01 -5.5412084e-01
  1.7371629e-01  1.3631743e-01  1.6446654e-01  2.7124006e-01
 -6.7972660e-02  6.5722942e-01 -1.3679255e-02  1.4928398e-02
 -5.8605385e-01  1.8562210e-01  8.0269173e-02  1.2528962e-04
  6.5358478e-01 -9.1295891e-02 -8.9641708e-01 -2.4489200e-02
  8.7145464e-03 -2.2659227e-01  4.3999436e-01 -1.2071882e-03
 -5.3521049e-01  1.4459050e-01 -9.2551613e-01 -1.7982474e-01
 -7.1112216e-01 -2.57243

In [20]:
result = test_tree.query(q_vec)
for name, dist in result:
    print(f"Name: {name} | Distance: {dist:.5f}")

Query in time [2.227ms]


Name: TAN CHEE HOON | Distance: 0.00000
Name: TAN CHEE HONG | Distance: 0.93064
Name: TAN HEE CHONG | Distance: 1.17114
Name: TAN CHEE YONG | Distance: 1.21273
Name: CHEN CHEE HUONG | Distance: 1.34736
Name: TAN WHEE HONG | Distance: 1.35105
Name: TAN PECK HOON | Distance: 1.35883
Name: OON CHEE WAH | Distance: 1.41322
Name: HON WEI TECK | Distance: 1.41708
Name: NEO CHIN CHER | Distance: 1.42421
Name: TAN CHEW CHEONG | Distance: 1.42911
Name: CHEN HONG | Distance: 1.45018
Name: CHEE KIAT HOE (XU JIEHE) | Distance: 1.45298
Name: CHEN CHEOW HIN | Distance: 1.46639
Name: TING PECK CHOR | Distance: 1.47220
Name: CHEW CHOR HIANG | Distance: 1.50341
Name: CHENG KWOK THIN | Distance: 1.51361
Name: CHEONG CHIEW YOON | Distance: 1.51814
Name: CHEO CHEANG ZHENG | Distance: 1.52094
Name: ANG HWEE CHOO | Distance: 1.53901


In [21]:
# Convenient Helper function for quering tree
def query_tree(query_string, annoy_tree):
    # encode query
#     q_vec = string2vec_encSum(sanitize_alpha(query_string))
    q_vec = c2v_model.vectorize_words([sanitize_alpha(query_string)])[0]
    result = annoy_tree.query(q_vec, search_k=10000)
    for name, dist in result:
        print(f"Name: {name} | Distance: {dist:.5f}")
    print("Brute-Forced Results:")
    result = annoy_tree.brute_force(q_vec)
    for name, dist in result:
        print(f"Name: {name} | Distance: {dist:.5f}")
    print("Control test with Levenshtein distance:")
    result = annoy_tree.control_query(sanitize_alpha(query_string))
    for name, dist in result:
        print(f"Name: {name} | Distance: {dist:.5f}")

In [22]:
query_tree("repblc hotels resors", test_tree)

Query in time [2.226ms]


Name: SREE LEKHA (S) PTE. LTD. | Distance: 1.69012
Name: HE-BREW COFFEE HOUSE PTE. LTD. | Distance: 1.82248
Name: EVERLITE PTE LTD | Distance: 1.83670
Name: SRI VEERA'S CURRY RESTAURANT PTE. LTD. | Distance: 1.85700
Name: VERRE PTE. LTD. | Distance: 1.87220
Name: SOLACE ENTERPRISE PTE LTD | Distance: 1.88055
Name: KER ENG HOE | Distance: 1.88564
Name: OVER EASY PTE. LTD. | Distance: 1.91238
Name: EXPERIENCE PROJECT SG PTE. LTD. | Distance: 1.91520
Name: PERFECT 12 PTE. LTD. | Distance: 1.91841
Name: NG POR KEE | Distance: 1.91844
Name: ROSWELL ENTERPRISE PTE LTD | Distance: 1.92400
Name: ELETEX ENTERPRISES PTE. LTD. | Distance: 1.97764
Name: SELETAR SEAFOOD CENTRE PTE LTD | Distance: 1.98097
Name: HONEYWELL AEROSPACE SINGAPORE PTE. LTD. | Distance: 1.98386
Name: PNG TECK SER | Distance: 1.99235
Name: SHELL EASTERN PETROLEUM (PTE) LTD | Distance: 1.99751
Name: SPICE CORNER LLP | Distance: 1.99942
Name: TAY SWEE KEONG | Distance: 2.01291
Name: LEE GEOK CHOO | Distance: 2.01347
Brute-Forc

Brute-Force in time [1087.327ms]


Name: SREE LEKHA (S) PTE. LTD. | Distance: 0.10455
Name: EXPERIENCE PROJECT SG PTE. LTD. | Distance: 0.10461
Name: ROSWELL ENTERPRISE PTE LTD | Distance: 0.10669
Name: VERRE PTE. LTD. | Distance: 0.10723
Name: SOLACE ENTERPRISE PTE LTD | Distance: 0.11023
Name: SRI VEERA'S CURRY RESTAURANT PTE. LTD. | Distance: 0.11052
Name: HE-BREW COFFEE HOUSE PTE. LTD. | Distance: 0.11074
Name: EVERLITE PTE LTD | Distance: 0.11364
Name: ELETEX ENTERPRISES PTE. LTD. | Distance: 0.11400
Name: SPICE CORNER LLP | Distance: 0.11410
Name: HONEYWELL AEROSPACE SINGAPORE PTE. LTD. | Distance: 0.11928
Name: NG POR KEE | Distance: 0.12231
Name: KER ENG HOE | Distance: 0.12788
Name: OVER EASY PTE. LTD. | Distance: 0.12896
Name: PERFECT 12 PTE. LTD. | Distance: 0.12911
Name: BEVERAGE 33 PTE.LTD. | Distance: 0.13230
Name: KA FENG ENTERPRISE PTE. LTD. | Distance: 0.13257
Name: EXTREMERS ADVENTURE PTE LTD | Distance: 0.13372
Name: LEE GEOK CHOO | Distance: 0.13495
Name: GERRES PTE. LTD. | Distance: 0.13506
Control 

Brute-Force with levenshtein [23002.286ms]


Name: REPUBLIC HOTELS & RESORTS LIMITED | Distance: 5.00000
Name: FURAMA HOTEL SINGAPORE PTE LTD? | Distance: 12.00000
Name: BAY HOTEL & RESORT PTE. LTD. | Distance: 12.00000
Name: REBECCA LOW SIEW YONG | Distance: 12.00000
Name: AMARA HOTEL PROPERTIES PTE LTD? | Distance: 13.00000
Name: RC HOTELS (PTE.) LTD | Distance: 13.00000
Name: ROYAL CATERING SERVICES PTE LTD | Distance: 13.00000
Name: LEGACY HOTEL PTE LTD | Distance: 13.00000
Name: AVANT HOTELS (SINGAPORE) PTE LTD? | Distance: 13.00000
Name: HARILELA HOTELS (SINGAPORE) PTE LTD? | Distance: 13.00000
Name: Republic of Singapore Yacht Club | Distance: 13.00000
Name: MEI JIA CHINESE RESTAURANT PTE LTD | Distance: 13.00000
Name: STRAND HOTEL PTE LTD | Distance: 13.00000
Name: RESTRANG PTE. LTD | Distance: 13.00000
Name: TEXAS CHICKEN RESTAURANT PTE. LTD. | Distance: 13.00000
Name: ES KOH PTE. LTD. | Distance: 13.00000
Name: SILOSO BEACH RESORT PTE. LTD. | Distance: 13.00000
Name: RAINTR33 HOTEL PTE. LTD. | Distance: 13.00000
Name: R

In [23]:
q1_vec = string2vec_encSum(sanitize_alpha("repblc hotels resors"))
q2_vec = string2vec_encSum(sanitize_alpha("REPUBLIC HOTELS & RESORTS LIMITED"))
print(distance.cosine(q1_vec, q2_vec))

0.04829701940586584


In [24]:
q1a_vec = c2v_model.vectorize_words([sanitize_alpha("repblc hotels resors")])[0]
q2a_vec = c2v_model.vectorize_words([sanitize_alpha("REPUBLIC HOTELS & RESORTS LIMITED")])[0]
print(distance.cosine(q1a_vec, q2a_vec))

0.20901548862457275
